In [1]:
import torch as th
import sys, os
import tqdm
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"
sys.path.insert(0, '../')
from sample_scripts.sample_utils.params_utils import get_params_set, load_params
from sample_scripts.sample_utils.vis_utils import plot_image


def load_face(set_, path):
    deca_params, _ = load_params(path=f"{path}/{set_}/", params_key=['shape', 'pose', 'exp', 'cam', 'light', 'tform', 'detail'])

    shape = []; pose = []; exp = []; cam = []; light = []; tform = []; detail = []
    for k, v in deca_params.items():
        
        shape.append(th.tensor(v['shape']))
        pose.append(th.tensor(v['pose']))
        exp.append(th.tensor(v['exp']))
        cam.append(th.tensor(v['cam']))
        light.append(th.tensor(v['light']))
        tform.append(th.tensor(v['tform']))
        detail.append(th.tensor(v['detail']))
    shape = th.stack(shape)
    pose = th.stack(pose)
    exp = th.stack(exp)
    cam = th.stack(cam)
    light = th.stack(light)
    tform = th.stack(tform)
    detail = th.stack(detail)
    concat_dict = {'shape':shape, 'pose':pose, 'exp':exp, 'cam':cam, 'light':light, 'tform':tform, 'detail':detail}
    return concat_dict, deca_params
    
deca_concat_dict_train, deca_id_dict_train = load_face('train', path='/data/mint/ffhq_256_with_anno/params_finale/')
deca_concat_dict_train_template, deca_id_dict_train_template = load_face('train_template', path='/data/mint/ffhq_256_with_anno/params_finale/')
deca_concat_dict_val, deca_id_dict_val = load_face('valid', path='/data/mint/ffhq_256_with_anno/params_finale/')
deca_concat_dict_val_template, deca_id_dict_val_template = load_face('valid_template', path='/data/mint/ffhq_256_with_anno/params_finale/')

Key=> shape : Filename=>/data/mint/ffhq_256_with_anno/params_finale//train/ffhq-train-shape-anno.txt
Key=> pose : Filename=>/data/mint/ffhq_256_with_anno/params_finale//train/ffhq-train-pose-anno.txt
Key=> exp : Filename=>/data/mint/ffhq_256_with_anno/params_finale//train/ffhq-train-exp-anno.txt
Key=> cam : Filename=>/data/mint/ffhq_256_with_anno/params_finale//train/ffhq-train-cam-anno.txt
Key=> light : Filename=>/data/mint/ffhq_256_with_anno/params_finale//train/ffhq-train-light-anno.txt
Key=> tform : Filename=>/data/mint/ffhq_256_with_anno/params_finale//train/ffhq-train-tform-anno.txt
Key=> detail : Filename=>/data/mint/ffhq_256_with_anno/params_finale//train/ffhq-train-detail-anno.txt
Key=> shape : Filename=>/data/mint/ffhq_256_with_anno/params_finale//train_template/ffhq-train-shape-anno.txt
Key=> pose : Filename=>/data/mint/ffhq_256_with_anno/params_finale//train_template/ffhq-train-pose-anno.txt
Key=> exp : Filename=>/data/mint/ffhq_256_with_anno/params_finale//train_template/f

# Verify for the same keys

In [2]:
assert deca_id_dict_val_template.keys() == deca_id_dict_val.keys()
assert deca_id_dict_train_template.keys() == deca_id_dict_train.keys()

# Verify for the values

In [3]:
import numpy as np
diff_dict = {}
for p in tqdm.tqdm(deca_concat_dict_train.keys()):
    diff_dict[p] = []
    for img_name in deca_id_dict_train.keys():
        a = deca_id_dict_train[img_name][p]
        b = deca_id_dict_train_template[img_name][p]
        diff_dict[p].append(np.allclose(a, b, rtol=1e-4, atol=1e-4))
        
for p in tqdm.tqdm(deca_concat_dict_train.keys()):
    print(f"Params : {p} => Equality = {np.all(diff_dict[p])}, Lengths = {len(diff_dict[p])}")

100%|██████████| 7/7 [00:00<00:00, 381.03it/s]

Params : shape => Equality = True, Lengths = 60000
Params : pose => Equality = True, Lengths = 60000
Params : exp => Equality = True, Lengths = 60000
Params : cam => Equality = True, Lengths = 60000
Params : light => Equality = True, Lengths = 60000
Params : tform => Equality = True, Lengths = 60000
Params : detail => Equality = True, Lengths = 60000


In [4]:
import numpy as np
diff_dict = {}
for p in tqdm.tqdm(deca_concat_dict_val.keys()):
    diff_dict[p] = []
    for img_name in deca_id_dict_val.keys():
        a = deca_id_dict_val[img_name][p]
        b = deca_id_dict_val_template[img_name][p]
        diff_dict[p].append(np.allclose(a, b, rtol=1e-4, atol=1e-4))
        
for p in tqdm.tqdm(deca_concat_dict_val.keys()):
    print(f"Params : {p} => Equality = {np.all(diff_dict[p])}, Lengths = {len(diff_dict[p])}")

100%|██████████| 7/7 [00:00<00:00, 1907.99it/s]

Params : shape => Equality = True, Lengths = 10000
Params : pose => Equality = True, Lengths = 10000
Params : exp => Equality = True, Lengths = 10000
Params : cam => Equality = True, Lengths = 10000
Params : light => Equality = True, Lengths = 10000
Params : tform => Equality = True, Lengths = 10000
Params : detail => Equality = True, Lengths = 10000
